## Setup
- Install Jinja templating library
- Define functions to create prompt and invoke LLM to generate query

In [ ]:
!pip install -q jinja2==3.1.*

In [ ]:
import os
import csv
import yaml
import json
import boto3
import utilities as u
import jinja2
from pathlib import Path
from typing import List, Union, Optional
from typing import Any as JsonType

from botocore.config import Config

config = Config(
   retries = {
      'max_attempts': 5,
      'mode': 'standard'
   }
)

session=boto3.Session()
bedrock_runtime = boto3.client(
    "bedrock-runtime", 
    region_name=u.AWS_REGION,
    config=config
)
jenv = jinja2.Environment(trim_blocks=True, lstrip_blocks=True)
model_id = "anthropic.claude-3-opus-20240229-v1:0"
temperature = 0.3
resources = Path.cwd() / "resources"
pfx = (Path.cwd() / "resources" / "prefixes.txt").read_text()
ground_truth = yaml.safe_load((resources / "ground-truth.yaml").read_text())
tips = yaml.safe_load((resources / "tips.yaml").read_text())


In [ ]:
prompt_template_json = yaml.safe_load((resources / "prompt.yaml").read_text())

def apply_jinja(json_blob: JsonType, **kwargs) -> JsonType:
    if isinstance(json_blob, str):
        return jenv.from_string(json_blob).render(
                        question=kwargs["question"],
                        tips=kwargs["tips"],
                        examples=kwargs["examples"])
    elif isinstance(json_blob, list):
        return [apply_jinja(x, **kwargs) for x in json_blob]
    elif isinstance(json_blob, dict):
        return {k: apply_jinja(v, **kwargs) for k, v in json_blob.items()}
    else:
        return json_blob

def generate_prompt(prompt_template_json: JsonType,
                    tips: List[str],
                    question: str,
                    few_shot_examples: List[dict]) -> JsonType:
    prompt_json = apply_jinja(prompt_template_json, 
                               question=question,
                               tips=tips,
                               examples=few_shot_examples)
    return prompt_json
    
def generate_sparql(prompt_template: JsonType,
                    tips: List[str],
                    question: str,
                    few_shot_examples: List[dict]) -> str:
    """
    Given a natural language question, use the LLM to transform that
    into a SPARQL query (using the prompt template) and return the query.
    """
    prompt = generate_prompt(prompt_template, tips, question, few_shot_examples)
    sys_prompt=prompt[0]['system']
    messages=prompt[1:]
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1024,
                "temperature": temperature,
                "messages": messages,
                "system": sys_prompt
            }
        ),
    )
    result = json.loads(response.get("body").read())
    output_list = result.get("content", [])
    sparql = "".join(output["text"] for output in output_list if output["type"] == "text")
    
    # get inside the <sparql> tag
    try:
        idx = sparql.index("<sparql>")
        sparql = sparql[idx+8:]
    except ValueError:
        pass
    try:
        idx = sparql.index("</sparql>")
        sparql = sparql[:idx]
    except ValueError:
        pass

    return sparql

## Run tests
- 4 of the 8 combos


In [ ]:
import time

def run_one_test(index, use_tips, use_few, folder_name):
    the_tips = tips if use_tips else []

    q=ground_truth[index]
    nlq=q['question']
    expected_sparql=q['SPARQL']
    error_msg=""
    gen_sparql=""
    res=None

    training_examples=[]
    if use_few:
        training_examples = ground_truth[:index] + ground_truth[index+1:]
        assert q["SPARQL"] not in {x["SPARQL"] for x in training_examples}

    generated_sparql = generate_sparql(prompt_template_json, the_tips, nlq, training_examples)
            
    sparql=f"""
        {pfx}

        {generated_sparql}

        LIMIT 20
    """
            
    try:
        res=u.execute_sparql(sparql, session) 
        if 'message' in res:
            error_msg=res['message'].replace("\n", " ")
        if not(folder_name is None):
            u.write_sparql_res(folder_name, str(index), nlq, expected_sparql, sparql, res, error_msg)
        else:
            print(nlq)
            print(sparql)
            print(res)

    except Exception as e:
        print(f"Error on {index}")
        print("Exception: {}".format(type(e).__name__))
        print("Exception message: {}".format(e))
        error_msg="Exception message: {}".format(e).replace("\n", " ")
        if not(folder_name is None):
            u.write_sparql_res(folder_name, str(index), nlq, q['SPARQL'], sparql, [], error_msg)

# In our testing our account had a throttle limit on Bedrock runtime model invocation.
# Besides using boto3 for retries and backoff, we also introduced a sleep between calls.
# Set this to -1 if you do NOT wish to sleep.
SLEEP_INTERVAL_SECS=70

def run_tests(folder_name, use_tips, use_few):

    folder=f"./{folder_name}" 
    if not(os.path.exists(folder) and os.path.isdir(folder)):
        os.mkdir(folder)

    for index, q in enumerate(ground_truth):
        if SLEEP_INTERVAL_SECS > 0:
            time.sleep(SLEEP_INTERVAL_SECS)
        print(f"{folder_name} {str(index)}")
        run_one_test(index, use_tips, use_few, folder_name)
        
def run_yourown_query(nlq):
    training_examples = ground_truth
            
    generated_sparql = generate_sparql(prompt_template_json, tips, nlq, training_examples) 
    sparql=f"""
        {pfx}

        {generated_sparql}

        LIMIT 20
    """
            
    res=u.execute_sparql(sparql, session) 
    
    print(generated_sparql)
    print(res)



In [ ]:
run_tests("implicit_zero", False, False)

run_tests("implicit_tips", True, False)

run_tests("implicit_few", False, True)

run_tests("implicit_few_tips", True, True)

In [ ]:
u.make_report("implicit_zero")
u.make_report("implicit_tips")
u.make_report("implicit_few")
u.make_report("implicit_few_tips")

## One-off queries


In [ ]:
run_one_test(5, True, True, None)

In [ ]:
run_yourown_query("What protein is in frogs")